In [ ]:
# 1. test the the complete data with all features on stacking model
# 2. tune fcm with all features and imputed result of stacking model


In [ ]:
import pandas as pd
import numpy as np
from hybrid_fcm_impute import HybridFCMImputer
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# import csv

X_train_complete = pd.read_csv('./Gas_Dataset/complete/X_train.csv')
y_train_complete = pd.read_csv('./Gas_Dataset/complete/y_train.csv')
X_test_complete = pd.read_csv('./Gas_Dataset/complete/X_test.csv')
y_test_complete = pd.read_csv('./Gas_Dataset/complete/y_test.csv')

In [ ]:
X_tr = X_train_complete[:1000]
X_te = X_test_complete[:1000]

## Metric Model: Stacking with complete dataset

In [ ]:
y_train_complete['Gas Class'] = y_train_complete['Gas Class'].apply(lambda x: x - 1)
y_test_complete['Gas Class'] = y_test_complete['Gas Class'].apply(lambda x: x - 1)


# check 

X_tr = X_train_complete[:1000]
X_te = X_test_complete[:1000]
y_tr = y_train_complete[:1000]
y_te = y_test_complete[:1000]

In [ ]:
def generate_stack_prediction(X_train, y_train, X_test, y_test):
    catboost_model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='MultiClass', verbose=False, random_state=42)
    xgboost_model = XGBClassifier(objective='multi:softmax', num_class=6, random_state=42)
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    catboost_model.fit(X_train, y_train)
    xgboost_model.fit(X_train, y_train)
    rf_model.fit(X_train, y_train)

    # 生成預測機率
    prob_catboost = catboost_model.predict_proba(X_test)
    prob_xgboost = xgboost_model.predict_proba(X_test)
    prob_rf = rf_model.predict_proba(X_test)

    # Stacking model使用機率當feature
    stacked_features = np.column_stack((prob_catboost, prob_xgboost, prob_rf))
    stacking_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
    stacking_model.fit(stacked_features, y_test)

    y_pred_stack = stacking_model.predict(stacked_features)
    accuracy_stack = accuracy_score(y_test, y_pred_stack)
    f1_stack = f1_score(y_test, y_pred_stack, average='macro')

    print("Stacking Model Test Accuracy:", accuracy_stack)
    print("Stacking Model Test F1 Score:", f1_stack)

In [ ]:
# generate_stack_prediction(X_train_complete, y_train_complete, X_test_complete, y_test_complete)


In [ ]:
# generate_stack_prediction(X_tr, y_tr, X_te, y_te)

## FCM with 0.25 missing

In [ ]:
# df = pd.concat([X_train_miss, X_test_miss])
df = X_train_complete
display(df)

In [ ]:
# tmp = pd.read_csv('./Gas_Dataset/X_train_top30.csv')
tmp = X_tr.copy()
tmp = tmp[:100]

In [ ]:
import pandas as pd
import numpy as np

def random_nan_half_rows(df, percentage):
    num_entries = df.size
    num_nans = int(num_entries * percentage / 100)
    half_rows = df.sample(frac=0.5)
    indices = np.random.choice(half_rows.size, num_nans, replace=False)
    half_rows_flattened = half_rows.to_numpy().flatten()
    half_rows_flattened[indices] = np.nan
    half_rows_with_nans = pd.DataFrame(half_rows_flattened.reshape(half_rows.shape), columns=half_rows.columns)
    df_with_nans = pd.concat([df.drop(half_rows.index), half_rows_with_nans])
    return df_with_nans

def count_complete_rows(df):
    complete_rows = df.dropna()
    num_complete_rows = len(complete_rows)
    return num_complete_rows

def missing_rate(df):
    num_missing = df.isna().sum().sum()
    total_entries = df.size
    missing_rate = num_missing / total_entries
    return missing_rate

df_incomplete = random_nan_half_rows(tmp, 25)
display(df_incomplete)

print(count_complete_rows(df_incomplete))
print(missing_rate(df_incomplete))


## Standard Regularization

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# create a StandardScaler object
scaler = StandardScaler()

# fit and transform the dataframe using the scaler
df_std = scaler.fit_transform(df_incomplete)

# convert the standardized dataframe to a pandas dataframe
df_std = pd.DataFrame(df_std, columns=df_incomplete.columns)

# print the standardized dataframe
print(df_std)


In [ ]:
from hybrid_fcm_impute import HybridFCMImputer
hfcmImputer = HybridFCMImputer(data = df_std, iter_num= 2)
after = hfcmImputer.impute()
display(after)

## Inverse transformation the dataset

In [ ]:
imputed = scaler.inverse_transform(after)
imputed